## 네이버 쇼핑 검색하기

In [1]:
from tkinter.tix import Tree
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [41]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
# 웹 싸이트 이동
driver.get("http://naver.com")
driver.implicitly_wait(time_to_wait=10) # 10초 대기(로딩 시간 필요)

try:
    # 쇼핑 메뉴 이동
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "a.nav.shop"))
    ).click()
    
    # 쇼핑몰 검색창 선택
    search = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input._searchInput_search_input_QXUFf"))
    )
    # 검색
    search.send_keys('삼겹살')
    search.send_keys(Keys.ENTER)
    time.sleep(5) # 검색 결과 확인 

    # 무산 스크롤
    before_h = driver.execute_script("return window.scrollY")
    while True:
        # 맨 아래로 스크롤 이동
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)

        after_h = driver.execute_script("return window.scrollY")
        if after_h == before_h:
            break
        before_h = after_h
    # 상품 정보 리스트
    items = driver.find_elements(By.CSS_SELECTOR, ".basicList_item__0T9JD")
    print(f'총 상품 수: {len(items)}')
    for i, item in enumerate(items):
        if i > 10:
            break

        name = item.find_element(By.CSS_SELECTOR, ".basicList_link__JLQJf").text
        link = item.find_element(By.CSS_SELECTOR, ".basicList_title__VfX3c > a").get_attribute("href")
        try:
            price = item.find_element(By.CSS_SELECTOR, ".price_num__S2p_v").text
        except:
            price = "판매중단"

        print(f'{i}번째 상품명: {name} / 가격: {price} / link: {link}')
finally:
    driver.quit()

총 상품 수: 46
0번째 상품명: 10mm내외 두툼한 돼지고기 삼겹살 500g / 가격: 7,500원 / link: https://adcr.naver.com/adcr?x=EOOQLQfmMmzNJkQQxsZcFv///w==khPnSQKgUGRaiuDYJkvdk5P9rmopjebHVusiouEuUMIJMaU0j2yl/+R7v9/XrSY66pS9jwJ6DoNjQV+f2ce/taMazWLZyru5Xn2m6rcKjJ2Rqlxh4A6004bS0j0RP27PWbBsDSK24TXp590mVyLVGR0PYGevWpo6vIW1SxVT24A5dsRXuQ/IgjhwCG+/FkXMDP0m3XU+6pOhOXGMIJy1ORxXMEtmDm1VmojlvZHlyYxB3sxmXp8xXk7EtLJX3IWFphWfXoAoK8Z72b8OsCBedhT2lPhvXqTQ7VO3V+i34BZfYPxLN9VTYdSBYphZSrdKd3ggI5MdtXtykWmPKfhkBjwL7+Hm3Nebvgnug03I7o+3B+CEb1aCyMv/2S46i93wPXENLXCt4j0im21XM2Y4+EaOq/iyGH2AysoDPU1i8TSxS1Ddm/n0wrhAyMsExjyYJiuHrdHxasdONMJ5bPdZJyT3rQwyYW6mJrTEq9pwc4gj7jrtzAKpU803fBDyzkiaLK2jUhs7Jx7lKfplXB75E+Pg6CdBgDqHiCYforyUqW/JlSCAwcaek20EzizHW0hc8aY0mJIV9V+PQ+R6Gm1mANO4jSh+7gGreIWg82tfxepuft35PwLFKFoG11EcDRAGwg8M0sYWh7i/n0if+culQBYysEHk5JSswROh4ba76ZClkGAey7NnQE9xpTRs8BVCfY1iZD4Fc0u2R3SGvwb7pEqn9mhTjqvjX74JeL3GZcl7JQg07LlTQAjiYkpPq/wkJYoAJrZsGsi6yuWVGprgkiVepRcFCzNYq4vS6l5CSeyj78w++yLKh2E1b+XY7vm50q6sR3okAFebUX0XVNbMxo3NNke/BqrXyiZDVx6pl5x3

### 사용자 함수 사용하기 

In [5]:
class search_item(object):
    def __init__(self, locator, search_item_nm) -> None:
        self.locator = locator
        self.search_item_nm = search_item_nm
        
    def __call__(self, driver):
        try:
            search = driver.find_element(*self.locator)
            search.send_keys(self.search_item_nm)
            search.send_keys(Keys.ENTER)
            return True 
        except:
            return False 

In [10]:
class select_items_info(object):
    def __init__(self, selector_dict) -> None:
        self.selector_dict = selector_dict

    def __call__(self, driver):
        try:
            # 무산 스크롤
            before_h = driver.execute_script("return window.scrollY")
            while True:
                # 맨 아래로 스크롤 이동
                driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
                time.sleep(1)

                after_h = driver.execute_script("return window.scrollY")
                if after_h == before_h:
                    break
                before_h = after_h

            # 상품 정보 리스트
            item_list = []
            items = driver.find_elements(By.CSS_SELECTOR, self.selector_dict['items'])
            print(f'총 상품 수: {len(items)}')
            for i, item in enumerate(items):
                if i > 10:
                    break

                name = item.find_element(By.CSS_SELECTOR, self.selector_dict['name']).text
                link = item.find_element(By.CSS_SELECTOR, self.selector_dict['link']).get_attribute("href")
                try:
                    price = item.find_element(By.CSS_SELECTOR, self.selector_dict['price']).text
                except:
                    price = "판매중단"

                item_info = f'{i}번째 상품명: {name} / 가격: {price} / link: {link}'
                item_list.append(item_info)
                print(item_info)

            return item_list
        except:
            return False 

In [11]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 웹 싸이트 이동
driver.get("http://naver.com")
driver.implicitly_wait(time_to_wait=10) # 10초 대기(로딩 시간 필요)

# 쇼핑 메뉴 이동
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "a.nav.shop"))
).click()

# 검색
wait = WebDriverWait(driver, 5)
search_result = wait.until(search_item((By.CSS_SELECTOR, "input._searchInput_search_input_QXUFf"), "삼겹살"))

if search_result:
    selector_dict = {
        "items": ".basicList_item__0T9JD",
        "name": ".basicList_link__JLQJf",
        "link": ".basicList_title__VfX3c > a",
        "price": ".price_num__S2p_v"
    }
    item_list = wait.until(select_items_info(selector_dict))
    print(len(item_list))


총 상품 수: 46
0번째 상품명: 삼겹살 목살 300g 단위 소분 포장 수입산 돼지고기 / 가격: 3,900원 / link: https://adcr.naver.com/adcr?x=XDQzYZNJlbeqj0MuD74Rrv///w==k2WVaPW6lUepRj9pz9lyGSZtohMwWEteyZG318Ri+xuLIpGsbpsfZxgcxRoOEKRcnF7uKifDEK+HioGEoJhIkZCAl6zUeYNox2oMC223UgriW02oXONS3yr84mL+SWea+RBn3BI5opLZVAlf/aQuKntEAGZG2AB9kGbuaNL7gXgqHHDYS0pc5Iq+/kyu1ckAlBQY+2AHj/tRCJzvIR7oDuCHZln5/tMwTdfIa8uu+kRRzDcIfarbJszkfHG18QoHsK+M5DEeqZb1P4acGDb/QAzEcohtaZlHAhW35kvq+yYbib0w3n79TMRRXjS2zgAOm3Gg4gJ4USN/ESDMEsSk9M8fW6ebv7aA9ToquraSvmlocpH2beBZR698qTOnZwJGLqDIK2v7rF5CUdcr4pkTizVMFII2KhlBHYYdKRCTpoVB2ki4nwizBj/VHQ4xEcpeCwhVEmZvZFeQ6kPmihEePuEx1nJlkhHJU16E/VzsYpYGyny53mj/wN0ET1zLQjOHB4aI9XKtaWatFhXpUZrZtQcQh4HaUycuvrTKgckuRlZCQ7git8iUgZot2o77rAzKBeEZ4wt45grW6C6eljI1d/L+RRxY3TRPvEJHYeDzz+f5X74JOkw3LoRz2/zJjNSzOx8tmiHDmotrqD1QoNOy7RtdHLM9uP5g+2J5fgJcN8BYsuuE+KBUF85u2MKEBybgDjn/n4eVESBmMcPhh51yl2KgYlH3H8uPQzwcZiTlXbIeJWdud96ZsL6eoh3xsZ2aC0zfCv/VfIXKQFMLdKOy7ojb/Ab9oHTjjc+TUFr/KooIfAz2u2gCMzoLWuOM0xI3klfVPGBY4sdM/K5Y+BoTIEH+CJtfuGzcM1ngiwz